## Library

In [1]:
! pip install -r requirements.txt

In [2]:
import random
import os
import copy
from collections import OrderedDict, defaultdict
from itertools import chain, islice, combinations
from time import time
from tqdm import tqdm

import dgl
from dgl.nn.pytorch import GraphConv
from dgl.nn.pytorch import SAGEConv

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import sklearn 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from networkx.algorithms.approximation.clique import maximum_independent_set as mis
import networkx as nx

from src import utils
from src import gnn
from src import instance

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# fix seed
SEED=0
utils.fix_seed(SEED)
torch_type=torch.float32

## CRA-PI-GNN

### Step1:  Load Problem and Set Hyperparameters

In [4]:
# Graph parameters
N, d, p, graph_type = 5000, 20, None, "reg"
nx_graph = nx.random_regular_graph(d=d, n=N, seed=SEED)
dgl_graph = dgl.from_networkx(nx_graph).to(device)
Q_mat = utils.qubo_dict_to_torch(nx_graph, instance.gen_q_dict_mis_sym(nx_graph,penalty=2)).to(device)

# GNN Architecture
in_feats = int(dgl_graph.number_of_nodes()**(0.5))
hidden_size=int(in_feats)
num_class=1
dropout=0.0
model=gnn.GCN_dev(in_feats, 
              hidden_size, 
              num_class, 
              dropout, 
              device).to(device)
embedding= nn.Embedding(dgl_graph.number_of_nodes(), 
                        in_feats
                       ).type(torch_type).to(device)

# Learning Parameters
num_epoch=int(1e+5)
lr=1e-4 
weight_decay=1e-2
tol=1e-4
patience=1000
vari_param=0
init_reg_param=-20
annealing_rate=1e-3
check_interval=1000
curve_rate=2                     

### Step 2: Define Loss Function

In [5]:
def loss(probs, reg_param, curve_rate=2):    
    probs_ = torch.unsqueeze(probs, 1)
    # cost function
    cost = (probs_.T @ Q_mat @ probs_).squeeze()
    # annealed term
    reg_term = torch.sum(1-(2*probs_-1)**curve_rate)
    return cost+reg_param*reg_term , cost, reg_term

### Set 3: Run PI-GNN solver

In [6]:
model, bit_string_PI, cost, reg_term, runtime = gnn.fit_model(model,
                                                         dgl_graph, 
                                                         embedding,
                                                         loss,
                                                         num_epoch=num_epoch,
                                                         lr=lr, 
                                                         weight_decay=weight_decay,
                                                         tol=tol, 
                                                         patience=patience, 
                                                         device=device,
                                                         annealing=False,
                                                         init_reg_param=0,
                                                         annealing_rate=0,
                                                         check_interval=check_interval,
                                                         curve_rate=curve_rate
                                                        )

【START】
【TRAIN EPOCH 0】LOSS 38718.742 COST 38718.742 REG 4937.979 PARAM 0.000
【TRAIN EPOCH 1000】LOSS 284.547 COST 284.547 REG 733.183 PARAM 0.000
【TRAIN EPOCH 2000】LOSS 42.714 COST 42.714 REG 291.718 PARAM 0.000
【TRAIN EPOCH 3000】LOSS 14.270 COST 14.270 REG 169.905 PARAM 0.000
【TRAIN EPOCH 4000】LOSS 6.171 COST 6.171 REG 112.166 PARAM 0.000
【TRAIN EPOCH 5000】LOSS 3.005 COST 3.005 REG 78.453 PARAM 0.000
【TRAIN EPOCH 6000】LOSS 1.558 COST 1.558 REG 56.588 PARAM 0.000
【TRAIN EPOCH 7000】LOSS 0.838 COST 0.838 REG 41.554 PARAM 0.000
【TRAIN EPOCH 8000】LOSS 0.461 COST 0.461 REG 30.850 PARAM 0.000
【TRAIN EPOCH 9000】LOSS 0.257 COST 0.257 REG 23.068 PARAM 0.000
【TRAIN EPOCH 10000】LOSS 0.145 COST 0.145 REG 17.332 PARAM 0.000
【TRAIN EPOCH 11000】LOSS 0.082 COST 0.082 REG 13.067 PARAM 0.000
【TRAIN EPOCH 12000】LOSS 0.047 COST 0.047 REG 9.876 PARAM 0.000
【TRAIN EPOCH 13000】LOSS 0.027 COST 0.027 REG 7.480 PARAM 0.000
【TRAIN EPOCH 14000】LOSS 0.015 COST 0.015 REG 5.674 PARAM 0.000
【TRAIN EPOCH 15000】LOSS 0.

### Step 3: Run CRA-PI-GNN solver

In [7]:
model, bit_string_CRA, cost, reg_term, runtime = gnn.fit_model(model,
                                                         dgl_graph, 
                                                         embedding,
                                                         loss,
                                                         num_epoch=num_epoch,
                                                         lr=lr, 
                                                         weight_decay=weight_decay,
                                                         tol=tol, 
                                                         patience=patience, 
                                                         device=device,
                                                         annealing=True,
                                                         init_reg_param=init_reg_param,
                                                         annealing_rate=annealing_rate,
                                                         check_interval=check_interval,
                                                         curve_rate=curve_rate
                                                        )

【START】
【TRAIN EPOCH 0】LOSS -5.333 COST 0.000 REG 0.267 PARAM -20.000
【TRAIN EPOCH 1000】LOSS -62621.617 COST 21374.662 REG 4420.857 PARAM -19.000
【TRAIN EPOCH 2000】LOSS -58284.879 COST 20586.074 REG 4381.720 PARAM -18.000
【TRAIN EPOCH 3000】LOSS -53962.191 COST 19744.613 REG 4335.694 PARAM -17.000
【TRAIN EPOCH 4000】LOSS -49676.172 COST 18860.611 REG 4283.549 PARAM -16.000
【TRAIN EPOCH 5000】LOSS -45436.805 COST 17932.307 REG 4224.607 PARAM -15.000
【TRAIN EPOCH 6000】LOSS -41253.844 COST 16956.037 REG 4157.849 PARAM -14.000
【TRAIN EPOCH 7000】LOSS -37137.938 COST 15928.104 REG 4082.003 PARAM -13.000
【TRAIN EPOCH 8000】LOSS -33100.734 COST 14844.926 REG 3995.472 PARAM -12.000
【TRAIN EPOCH 9000】LOSS -29155.059 COST 13703.106 REG 3896.197 PARAM -11.000
【TRAIN EPOCH 10000】LOSS -25315.529 COST 12499.744 REG 3781.527 PARAM -10.000
【TRAIN EPOCH 11000】LOSS -21599.461 COST 11233.095 REG 3648.062 PARAM -9.000
【TRAIN EPOCH 12000】LOSS -18027.859 COST 9903.521 REG 3491.423 PARAM -8.000
【TRAIN EPOCH 13000

In [8]:
size_mis_CRA, _, number_violation = utils.postprocess_gnn_mis(bit_string_CRA, nx_graph)
size_mis_PI, _, number_violation = utils.postprocess_gnn_mis(bit_string_PI, nx_graph)
print(f"Independent set size: (CRA) {size_mis_CRA.item()}, (PI) {size_mis_PI.item()}")

Independent set size: (CRA) 853, (PI) 0
